### Преобразование базового датасета в датасет для тренировки выделения связей

У нас есть датасет с записями вида

```
{
    "scene": 
        {
            "location": "тренажерный зал", 
            "objects": [
                {"гантель": ["тяжёлая"]}, 
                {"штанга": ["тяжелая", "металлическая"]}, 
                {"скамья": ["деревянная", "тяжелая"]}
            ], 
            "relations": [
                ["гантель", "на", "скамья"], 
                ["гантель", "рядом с", "штанга"]
            ]
        }, 
     "description": "Тяжелая гантель лежит на деревянной тяжелой скамье рядом с тяжелой металлической штангой."
}
```

будем преобразовывать его в новый датасет для обучения T5ru выделения не только связей между объектами но и типов связи (собственно если не нужно определять тип связи то подошел бы и BERT в качестве классификатора на парах)


структура датасета

```
{
    "description": "Тяжелая гантель лежит на деревянной тяжелой скамье рядом с тяжелой металлической штангой."
    "relation": ["гантель", "на", "скамья"],
    "target": "на"
}

{
    "description": "Тяжелая гантель лежит на деревянной тяжелой скамье рядом с тяжелой металлической штангой."
    "relation": ["гантель", "на", "штанга"],
    "target": "нет связи"
}

```

в датасет помещаем примерно в равных количествах верные связи, отсутствующие связи.
Учим модель угадывать связь. 


неверные связи (связь есть но неверно выделена) - такое пока не делаем, думаю что в первом приближении пусть хоть как-то определяет тип связи, сосредоточимся на "есть связь" - "нет связи". 


Такой подход нам позволяет учиться на правильно выделенных объектах а в проде если объекты выделены не все или неверно делать предикты только на парах объектов которые выделила первая модель (модель выделяющая объекты и признаки)

То есть мы учим модели независимо - одну выделять объекты и их признаки другую - по выделенным (возможно частично или ошибочно) признакам выделять связи и определять их характер

**Замечание**

мы даем в датасет все верные связи (N), N/2 инвертированных с меткой "нет связи" и еще N негативных с меткой "нет связи". Именно такие пропорции чтобы модель не отмахивалась и не ставила "нет связи" вообще везде. 

In [19]:
import json
import random
from itertools import permutations
from pathlib import Path
from typing import List, Tuple


In [67]:
def load_jsonl(path: Path) -> List[dict]:
    with path.open(encoding="utf-8") as f:
        return [json.loads(line) for line in f]

def extract_object_names(object_list: List[dict]) -> List[str]:
    return [list(obj.keys())[0] for obj in object_list]

def build_input(description: str, obj1: str, obj2: str) -> str:
    return json.dumps([obj1, obj2], ensure_ascii=False)

def get_relations_lookup(relations: List[Tuple[str, str, str]]) -> dict:
    return {(o1, o2): rel for o1, rel, o2 in relations}

def build_directional_dataset_from_jsonl_file(path: Path) -> List[dict]:
    """Build dataset with correct, inverted, and negative object pairs."""
    data = load_jsonl(path)
    results = []

    for record in data:
        desc = record["description"]
        scene = record["scene"]
        object_names = extract_object_names(scene["objects"])

        if len(object_names) < 2:
            continue

        true_relations = scene.get("relations", [])

        rel_lookup = get_relations_lookup(true_relations)
        true_pairs = list(rel_lookup.keys())
        all_possible_pairs = list(permutations(object_names, 2))

        # 1. Истинные пары
        for obj1, obj2 in true_pairs:
            relation = [obj1, obj2]
            target = rel_lookup[(obj1, obj2)]
            results.append({"description": desc, "relation": relation, "target": target})

        N = len(true_pairs)

        # 2. Инвертированные пары — подаём в обратном порядке, метка "нет связи"
        inverted_candidates = [(b, a) for a, b in true_pairs if (b, a) not in rel_lookup]
        sampled_inverted = random.sample(inverted_candidates, min(N // 2, len(inverted_candidates)))
        for obj1, obj2 in sampled_inverted:
            relation = [obj1, obj2] 
            results.append({"description": desc, "relation": relation, "target": "нет связи"})

        # 3. Пары без связи вообще
        negative_candidates = [
            (a, b) for a, b in all_possible_pairs
            if (a, b) not in rel_lookup and (b, a) not in rel_lookup
        ]
        sampled_negatives = random.sample(negative_candidates, min(N, len(negative_candidates)))
        for obj1, obj2 in sampled_negatives:
            relation = [obj1, obj2]
            results.append({"description": desc, "relation": relation, "target": "нет связи"})

    return results

In [69]:
#build_directional_dataset_from_jsonl_file(Path("dataset_syntetic_v5_spacial_only/src/dataset_spacial_batch_000.jsonl"))

In [70]:
def build_directional_dataset_from_all_batches(
    input_dir: Path,
    output_dir: Path,
    pattern: str = "dataset_spacial_batch_*.jsonl",
    output_prefix: str = "spatial_relations_batch_"
) -> int:
    
    output_dir.mkdir(parents=True, exist_ok=True)
    total_records = 0

    for idx, path in enumerate(sorted(input_dir.glob(pattern))):
        batch_data = build_directional_dataset_from_jsonl_file(path)
        total_records += len(batch_data)

        output_path = output_dir / f"{output_prefix}{idx:03d}.jsonl"
        with output_path.open("w", encoding="utf-8") as f:
            for record in batch_data:
                json.dump(record, f, ensure_ascii=False)
                f.write("\n")

    return total_records


In [72]:
build_directional_dataset_from_all_batches(
    Path("dataset_syntetic_v5_spacial"),
    Path("dataset_syntetic_v5_spacial_only")
)

21671

### Результат - получили датасет из 21671 записей (текст+ пара+связь)